In [50]:
import geopandas as gpd
import vaex
import pandas as pd
import numpy as np

In [96]:
gdf_lote = gpd.read_file('lotes_agregados_por_ano/1995/SIRGAS_SHP_LOTES_90_VILA_MARIANA_IPTU_1995.gpkg').drop_duplicates(subset=['sqlc']).set_index('sqlc')
gdf_lote = gdf_lote[gdf_lote.is_valid]

In [134]:
df_iptu = vaex.open('data/por_distritos/IPTU-1995-2022-agrupados-por-sqlc-90-vila-mariana.hdf5').to_pandas_df().set_index('sqlc')
df_iptu_ano = df_iptu[df_iptu.ano == 2017]

In [135]:
df_iptu_ano.shape, gdf_lote.shape

((16341, 29), (15209, 3))

In [136]:
np.unique(df_iptu_ano.index.duplicated()), np.unique(gdf_lote.index.duplicated(), return_counts=True)

(array([False]), (array([False]), array([15209])))

In [156]:
## Lotes existentes no IPTU no ano escolhido e com geometria do lote
gdf_lote.join(df_iptu_ano, how='inner')

,sq,agregado,geometry,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos terrenos-lotes,Área Total Ocupada,...,Residencial,Comercial,Serviços,Industrial,Outros,Percentual de Uso Residencial,Percentual de Uso Comercial,Percentual de Uso Serviços,Percentual de Uso Industrial,Percentual de Uso Outros
sqlc,,,,,,,,,,,,,,,,,,,,,
037083000006,037083,False,"POLYGON ((332904.860 7390044.150, 332900.337 7...",2017,131,131,267.328244,7812.0,1023372,589500,...,35020,0,0,0,0,1.0,0.0,0.0,0.0,0.0
037083000004,037083,False,"POLYGON ((332838.721 7390116.947, 332839.550 7...",2017,121,121,92.537190,1943.0,235103,72600,...,11197,0,0,0,0,1.0,0.0,0.0,0.0,0.0
038053000006,038053,False,"POLYGON ((333182.830 7391765.482, 333183.358 7...",2017,4,4,95.000000,300.0,1200,916,...,380,0,0,0,0,1.0,0.0,0.0,0.0,0.0
036007000013,036007,False,"POLYGON ((331713.726 7392268.575, 331715.362 7...",2017,72,72,60.916667,820.0,59040,16560,...,4386,0,0,0,0,1.0,0.0,0.0,0.0,0.0
039077000008,039077,False,"POLYGON ((333454.629 7390558.543, 333443.842 7...",2017,24,24,147.000000,675.0,16200,5712,...,3528,0,0,0,0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
042055002100,042055,False,"POLYGON ((333332.478 7389371.809, 333356.081 7...",2017,1,0,NaN,105.0,105,65,...,115,0,0,0,0,1.0,0.0,0.0,0.0,0.0
037003002900,037003,False,"POLYGON ((332097.007 7391470.434, 332098.349 7...",2017,1,0,NaN,96.0,96,76,...,0,0,152,0,0,0.0,0.0,1.0,0.0,0.0
039139019300,039139,False,"POLYGON ((334593.643 7391134.309, 334593.340 7...",2017,1,0,NaN,141.0,141,54,...,161,0,0,0,0,1.0,0.0,0.0,0.0,0.0


In [162]:
# Lotes sem geometria que foram desmembrados ou remembrados e pprecisam ser agregados
lotes_sg = df_iptu_ano.join(gdf_lote, how='left').sq.isna()
df_lotes_sg = df_iptu_ano[lotes_sg].reset_index()

In [174]:
# list(df_lotes_sg.columns)

In [173]:
df_lotes_sg.sqlc = df_lotes_sg.sqlc.str[:6] + '000000'
df_lotes_sg_group = df_lotes_sg.groupby('sqlc').agg(
    {'ano':'first',
    'Quantidade de Unidades':'sum',
    'Quantidade de Unidades Condominiais':'sum',
    'Tamanho Médio da Unidade Condominial':'mean',
    'Tamanho médio dos Terrenos':'mean',
    'Área Total dos terrenos-lotes':'sum',
    'Área Total Ocupada':'sum',
    'Área Total Construída':'sum',
    'Valor Total dos Terrenos':'sum',
    'Valor Total das Construções':'sum',
    'CA médio':'mean',
    'TO médio':'mean',
    'CA médio em lotes condominiais':'mean',
    'TO médio em lotes condominiais':'mean',
    'CA médio em lotes não condominiais':'mean',
    'TO médio em lotes não condominiais':'mean',
    'Comprimento Médio da Testada':'mean',
    'Número médio de Pavimentos':'mean',
    'Fator de obsolecência médio':'mean',
    'Residencial':'mean',
    'Comercial':'mean',
    'Serviços':'mean',
    'Industrial':'mean',
    'Outros':'mean',
    'Percentual de Uso Residencial':'mean',
    'Percentual de Uso Comercial':'mean',
    'Percentual de Uso Serviços':'mean',
    'Percentual de Uso Industrial':'mean',
    'Percentual de Uso Outros':'mean'}
)
# df_lotes_sg.set_index('sqlc')

In [175]:
# Agora com as geometrias
gdf_lote.join(df_lotes_sg_group, how='inner')

,sq,agregado,geometry,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos terrenos-lotes,Área Total Ocupada,...,Residencial,Comercial,Serviços,Industrial,Outros,Percentual de Uso Residencial,Percentual de Uso Comercial,Percentual de Uso Serviços,Percentual de Uso Industrial,Percentual de Uso Outros
sqlc,,,,,,,,,,,,,,,,,,,,,
034077000000,034077,True,"MULTIPOLYGON (((334114.300 7391437.974, 334114...",2017,64,62,120.419355,598.666667,95492,76296,...,2596.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
034079000000,034079,True,"MULTIPOLYGON (((334407.395 7391514.851, 334406...",2017,60,59,94.389831,677.500000,57325,36440,...,2784.500000,272.5,0.000000,0.0,0.0,0.500000,0.5,0.000000,0.0,0.0
034080000000,034080,True,"POLYGON ((334316.102 7391378.138, 334311.496 7...",2017,4,4,152.500000,329.000000,1316,600,...,610.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
034081000000,034081,True,"MULTIPOLYGON (((334483.258 7391521.194, 334483...",2017,79,79,86.974684,1516.000000,119764,73470,...,6871.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
034082000000,034082,True,"MULTIPOLYGON (((334551.441 7391385.448, 334543...",2017,20,19,90.842105,542.500000,12767,7331,...,863.000000,0.0,228.000000,0.0,0.0,0.500000,0.0,0.500000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
042188000000,042188,True,"MULTIPOLYGON (((334297.921 7389458.557, 334304...",2017,15,0,NaN,171.200000,2568,1246,...,125.333333,0.0,36.066667,0.0,0.0,0.857143,0.0,0.142857,0.0,0.0
042189000000,042189,True,"POLYGON ((334317.077 7389590.257, 334312.635 7...",2017,2,0,NaN,149.000000,298,116,...,113.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
042190000000,042190,True,"MULTIPOLYGON (((334422.506 7389680.953, 334421...",2017,3,0,NaN,448.000000,1344,0,...,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN
